### Import Statements

In [24]:
import geopandas as gpd

import cuxfilter
from cuxfilter.layouts import feature_and_five_edge, feature_and_double_base, feature_and_base
import cudf
import numpy as np

import holoviews as hv
import pandas as pd

### Importing HouseHold Distance Data

In [5]:
df_charleston = pd.read_parquet('/hpc/group/codeplus22-vis/infousa_copy/distances_charleston_final.parquet')
df_charleston = df_charleston[df_charleston['distance_category'] != 4]
df_charleston

,has_child,age_code,lat_3857,lon_3857,tank_type,diameter,distance_m,distance_mi,distance_category,is_elderly
0,2,C,-8.897856e+06,3.865676e+06,external_floating_roof_tank,10.8,2472.401421,1.536279,3,2
1,2,M,-8.897856e+06,3.865676e+06,external_floating_roof_tank,10.8,2472.401421,1.536279,3,1
2,2,L,-8.897856e+06,3.865676e+06,external_floating_roof_tank,10.8,2472.401421,1.536279,3,1
3,1,D,-8.897856e+06,3.865676e+06,external_floating_roof_tank,10.8,2472.401421,1.536279,3,2
4,2,M,-8.897856e+06,3.865676e+06,external_floating_roof_tank,10.8,2472.401421,1.536279,3,1
...,...,...,...,...,...,...,...,...,...,...
245345,0,None,-8.898830e+06,3.872435e+06,spherical_tank,48.6,NaN,35.000000,0,0
245346,0,None,-8.898897e+06,3.872415e+06,spherical_tank,51.6,NaN,35.000000,0,0
245347,0,None,-8.898822e+06,3.872099e+06,closed_roof_tank,12.6,NaN,35.000000,0,0
245348,0,None,-8.898841e+06,3.872089e+06,closed_roof_tank,12.6,NaN,35.000000,0,0


### Defining Charts

In [6]:
label_map_distance = {0: 'Tank', 1: '0.5 miles away', 
             2: '1 mile away', 3: '5 miles away'}

label_map_elderly = {0: 'Tank', 1: 'Elderly', 
             2: 'Not Elderly'}

label_map_children = {0: 'Tank', 1: 'Children', 
             2: 'No Children'}

colors = ['#05c1ff', '#ff0000', '#ff00a4', '#a11aeb']

### Transforming to cuxfilter dataframe

In [8]:
cdf = cudf.DataFrame.from_pandas(df_charleston) 

In [9]:
cux_df = cuxfilter.DataFrame.from_dataframe(cdf) 

### Making Cuxfilter Charts

In [34]:
points = cuxfilter.charts.scatter(x='lat_3857', y='lon_3857', pixel_shade_type='linear', color_palette = colors, aggregate_fn = 'max', aggregate_col = 'distance_category', tile_provider="CartoDark", title = 'Households in Charleston County in Close Proximity to Tanks',
                                   x_range=(-13825798.514061378,-7542228.134036879), y_range=(2819963.842141629,6272600.009501693), legend = True)

distance_category = cuxfilter.charts.multi_select('distance_category', label_map=label_map_distance)

age = cuxfilter.charts.multi_select('is_elderly', label_map=label_map_elderly)

children = cuxfilter.charts.multi_select('has_child', label_map=label_map_children)

distance_slider = cuxfilter.charts.range_slider('distance_mi')

In [35]:
d = cux_df.dashboard([points, distance_slider], sidebar = [distance_category, age, children], layout = cuxfilter.layouts.feature_and_base, theme = cuxfilter.themes.rapids) 

### Displaying interactive dashboard

In [37]:
d.show()
d.app(sidebar_width=290) # run the dashboard within the notebook ce

Dashboard running at port 34169


Row(sizing_mode='stretch_both')
    [0] Column(sizing_mode='stretch_height', width=290)
        [0] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='Datapoints Selected')
            [0] Column(sizing_mode='stretch_both')
                [0] Number(css_classes=['indicator'], font_size='18pt', format='{value:,}', sizing_mode='stretch_width', value=151090)
                [1] Progress(sizing_mode='stretch_width', value=100)
        [1] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='distance_category_multi_s..., width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, '0.5 miles awa...}, sizing_mode='stretch_both', value=[''], width=400)
        [2] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='is_elderly_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, 'Elderly': 1, ...}, sizing_mode='stretch_both', value=[''], width=400)
        [3] Card(collapsible=False, header_css_classes=['card-header', ...], sizing_mode='scale_width', title='has_child_multi_select', width=400)
            [0] MultiSelect(height=200, options={'Tank': 0, 'Children': 1,...}, sizing_mode='stretch_both', value=[''], width=400)
    [1] GridStack(allow_drag=False, allow_resize=False, ncols=11, nrows=6, sizing_mode='stretch_both')
        [0] Card(sizing_mode='stretch_both', title='Households in C..., width=800)
            [0] HoloViews(DynamicMap, height=400, sizing_mode='stretch_both')
        [1] Card(sizing_mode='stretch_both', title='distance_mi_range_slider'..., width=400)
            [0] RangeSlider(end=35.0, sizing_mode='stretch_both', start=0.013255532840467104, value=(0.013255532840467104, ..., value_end=35.0, value_start=0.013255532840467104)